In [ ]:
import pandas as pd

origem = "../data/votacao_candidato_munzona_2024_BRASIL.csv"
saida  = "../data/votacao_prefeitos_filtrada.csv"

# Colunas que deseja manter
colunas = [
    "NR_TURNO", "SG_UF", "NM_MUNICIPIO", "SG_UE", "DS_CARGO",
    "NM_CANDIDATO", "NM_URNA_CANDIDATO", "SG_PARTIDO", "DS_COMPOSICAO_COLIGACAO",
    "NM_PARTIDO", "QT_VOTOS_NOMINAIS", "QT_VOTOS_NOMINAIS_VALIDOS",
    "NR_ZONA", "DS_SIT_TOT_TURNO"
]

# Leitura em blocos
chunksize = 200_000            # ajuste conforme a memória disponível
primeiro_bloco = True

for chunk in pd.read_csv(
        origem,
        sep=";",               # caso o separador seja ';'
        encoding="latin1",
        usecols=colunas,
        chunksize=chunksize
):
    # mantém apenas votos para prefeito
    chunk = chunk[chunk["DS_CARGO"] == "Prefeito"]

    # grava em modo de append (adicionando blocos)
    chunk.to_csv(
        saida,
        index=False,
        mode="w" if primeiro_bloco else "a",
        header=primeiro_bloco,
        encoding="utf-8-sig"
    )
    primeiro_bloco = False


In [17]:
# Contar quantidade de municipios únicos de "../data/votacao_candidato_munzona_2024_BRASIL.csv"
municipios_unicos = pd.read_csv(
    origem,
    sep=";",
    encoding="latin1",
    usecols=["SG_UF", "NM_MUNICIPIO"],
    chunksize=chunksize
)
municipios_unicos = pd.concat(municipios_unicos)
municipios_unicos = municipios_unicos.drop_duplicates(subset=["SG_UF", "NM_MUNICIPIO"])
municipios_unicos_count = municipios_unicos.shape[0]
print(f"Quantidade de municípios únicos: {municipios_unicos_count}")

Quantidade de municípios únicos: 5568


In [19]:
# Contar quantidade de municipios únicos
municipios_filtrados = pd.read_csv(
    saida,
    usecols=["SG_UF", "NM_MUNICIPIO"],
    encoding="utf-8-sig"
)
municipios_filtrados = municipios_filtrados.drop_duplicates(subset=["SG_UF", "NM_MUNICIPIO"])
municipios_filtrados_count = municipios_filtrados.shape[0]
print(f"Quantidade de municípios únicos filtrados: {municipios_filtrados_count}")

Quantidade de municípios únicos filtrados: 5561


In [14]:
import pandas as pd

origem   = "../data/votacao_candidato_munzona_2024_BRASIL.csv"
filtrado = "../data/votacao_prefeitos_filtrada.csv"

# 1) Municípios do arquivo original
mun_origem = set()
for chunk in pd.read_csv(
        origem,
        sep=";",              # separador do original
        encoding="latin1",    # encoding do original
        usecols=["NM_MUNICIPIO"],
        chunksize=200_000
    ):
    mun_origem.update(chunk["NM_MUNICIPIO"].unique())

# 2) Municípios do arquivo filtrado
mun_filt = set()
for chunk in pd.read_csv(
        filtrado,
        encoding="utf-8-sig",
        usecols=["NM_MUNICIPIO"],
        chunksize=200_000
    ):
    mun_filt.update(chunk["NM_MUNICIPIO"].unique())

# 3) Diferença: presentes no original mas não no filtrado
diff = mun_origem - mun_filt

print(f"{len(diff)} municípios ausentes no filtrado:")
for nome in sorted(diff):
    print(nome)


7 municípios ausentes no filtrado:
AMERICANO DO BRASIL
ANAJATUBA
BARRA DOS COQUEIROS
CEDRO DE SÃO JOÃO
GUIMARÃES
NOSSA SENHORA DO SOCORRO
SANTANA DO MARANHÃO


In [18]:
import pandas as pd

origem   = "../data/votacao_candidato_munzona_2024_BRASIL.csv"
filtrado = "../data/votacao_prefeitos_filtrada.csv"
chunksize = 200_000

# 1) Constrói o set de pares (UF, município) do arquivo original
mun_origem = set()
for chunk in pd.read_csv(
        origem,
        sep=";",
        encoding="latin1",
        usecols=["SG_UF", "NM_MUNICIPIO"],
        chunksize=chunksize
    ):
    pairs = zip(chunk["SG_UF"].str.strip(), chunk["NM_MUNICIPIO"].str.strip())
    mun_origem.update(pairs)

# 2) Constrói o set de pares (UF, município) do arquivo filtrado
mun_filt = set()
for chunk in pd.read_csv(
        filtrado,
        encoding="utf-8-sig",
        usecols=["SG_UF", "NM_MUNICIPIO"],
        chunksize=chunksize
    ):
    pairs = zip(chunk["SG_UF"].str.strip(), chunk["NM_MUNICIPIO"].str.strip())
    mun_filt.update(pairs)

# 3) Diferença
diff = mun_origem - mun_filt
print(f"Total originais: {len(mun_origem)}")
print(f"Total filtrados: {len(mun_filt)}")
print(f"Municípios ausentes: {len(diff)}\n")
for uf, mun in sorted(diff):
    print(f"{uf} — {mun}")


Total originais: 5568
Total filtrados: 5561
Municípios ausentes: 7

GO — AMERICANO DO BRASIL
MA — ANAJATUBA
MA — GUIMARÃES
MA — SANTANA DO MARANHÃO
SE — BARRA DOS COQUEIROS
SE — CEDRO DE SÃO JOÃO
SE — NOSSA SENHORA DO SOCORRO


In [20]:
import pandas as pd

# caminhos
origem   = "../data/votacao_candidato_munzona_2024_BRASIL.csv"
filtrado = "../data/votacao_prefeitos_filtrada.csv"

# lista dos 7 municípios “ausentes”
missing = [
    'AMERICANO DO BRASIL',
    'ANAJATUBA',
    'BARRA DOS COQUEIROS',
    'CEDRO DE SÃO JOÃO',
    'GUIMARÃES',
    'NOSSA SENHORA DO SOCORRO',
    'SANTANA DO MARANHÃO'
]

chunksize = 200_000
# colunas que vamos precisar
cols = ["SG_UF", "NM_MUNICIPIO", "DS_CARGO",
        "NR_TURNO", "QT_VOTOS_NOMINAIS", "QT_VOTOS_NOMINAIS_VALIDOS",
        "NR_ZONA", "DS_SIT_TOT_TURNO"]

# 1) Mostrar alguns registros brutos do original para esses municípios
print("=== Registros brutos no original ===")
for chunk in pd.read_csv(origem, sep=";", encoding="latin1",
                         usecols=cols, chunksize=chunksize):
    sel = chunk[chunk["NM_MUNICIPIO"].isin(missing)]
    if not sel.empty:
        print(sel.head(5))  # mostra as 5 primeiras linhas de cada chunk
print()

# 2) Contagem de registros por cargo, no original
print("=== Contagem por cargo (original) ===")
parts = []
for chunk in pd.read_csv(origem, sep=";", encoding="latin1",
                         usecols=cols, chunksize=chunksize):
    df = chunk[chunk["NM_MUNICIPIO"].isin(missing)]
    if not df.empty:
        parts.append(
            df.groupby(["NM_MUNICIPIO", "DS_CARGO"])
              .size()
              .reset_index(name="qt_registros")
        )
if parts:
    cnt_orig = pd.concat(parts)
    cnt_orig = (cnt_orig
                .groupby(["NM_MUNICIPIO", "DS_CARGO"])["qt_registros"]
                .sum()
                .reset_index())
    print(cnt_orig)
print()

# 3) Quantas linhas com DS_CARGO == 'Prefeito'
print("=== Quantos Prefeito no original ===")
if not parts:
    print("  nenhum registro encontrado")
else:
    pref = cnt_orig[cnt_orig["DS_CARGO"] == "Prefeito"]
    print(pref if not pref.empty else "  zero prefeitos")
print()

# 4) Conferir no CSV já filtrado se escapou algum
print("=== Registros no filtrado (prefeitos) ===")
df_f = pd.read_csv(filtrado, encoding="utf-8-sig",
                   usecols=["SG_UF", "NM_MUNICIPIO"])
fil = (df_f[df_f["NM_MUNICIPIO"].isin(missing)]
       .groupby(["SG_UF", "NM_MUNICIPIO"])
       .size()
       .reset_index(name="qt_registros"))
print(fil if not fil.empty else "  nenhum registro no filtrado")
print()

# 5) Examinar comprimento e bytes hex para detectar espaços ou acentos “esquisitos”
print("=== Bytes de cada nome (origem) ===")
seen = set()
for chunk in pd.read_csv(origem, sep=";", encoding="latin1",
                         usecols=["NM_MUNICIPIO"], chunksize=chunksize):
    df = chunk[chunk["NM_MUNICIPIO"].isin(missing)]
    for nm in df["NM_MUNICIPIO"].unique():
        if nm not in seen:
            b = nm.encode("latin1")
            print(f"{nm!r}: len={len(nm)} bytes_hex={b.hex()}")
            seen.add(nm)
    if len(seen) == len(missing):
        break


=== Registros brutos no original ===
      NR_TURNO SG_UF         NM_MUNICIPIO  NR_ZONA  DS_CARGO  \
1011         1    MA            GUIMARÃES       30  Vereador   
3420         1    GO  AMERICANO DO BRASIL       34  Vereador   
3999         1    MA            GUIMARÃES       30  Vereador   
4339         1    SE  BARRA DOS COQUEIROS        2  Vereador   
4340         1    SE  BARRA DOS COQUEIROS        2  Vereador   

      QT_VOTOS_NOMINAIS  QT_VOTOS_NOMINAIS_VALIDOS  DS_SIT_TOT_TURNO  
1011                228                        228  ELEITO POR MÉDIA  
3420                232                        232  ELEITO POR MÉDIA  
3999                293                        293  ELEITO POR MÉDIA  
4339                  0                          0        NÃO ELEITO  
4340                 31                         31        NÃO ELEITO  
        NR_TURNO SG_UF              NM_MUNICIPIO  NR_ZONA  DS_CARGO  \
209973         1    MA       SANTANA DO MARANHÃO       51  Vereador   
214468    

In [21]:
import pandas as pd

origem   = "../data/votacao_candidato_munzona_2024_BRASIL.csv"
missing = [
    'AMERICANO DO BRASIL',
    'ANAJATUBA',
    'BARRA DOS COQUEIROS',
    'CEDRO DE SÃO JOÃO',
    'GUIMARÃES',
    'NOSSA SENHORA DO SOCORRO',
    'SANTANA DO MARANHÃO'
]

# verifica cargos únicos desses municípios
cargos = set()
for chunk in pd.read_csv(origem,
                         sep=";",
                         encoding="latin1",
                         usecols=["NM_MUNICIPIO", "DS_CARGO"],
                         chunksize=200_000):
    sel = chunk[chunk["NM_MUNICIPIO"].isin(missing)]
    cargos.update(sel["DS_CARGO"].str.strip().unique())
    
print("Cargos encontrados nesses 7 municípios:", cargos)
# deve imprimir apenas {'Vereador'}


Cargos encontrados nesses 7 municípios: {'Vereador'}


In [ ]:
# Execute src\data_pipeline.py code

FileNotFoundError: [Errno 2] No such file or directory: 'data/emendas.csv'